In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/nagarjuna843/XGBoost/main/churn.csv')
print(df.head())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
inputs=df.drop(['RowNumber','CustomerId','Surname','Exited'],axis='columns')
target=df['Exited']
print(inputs.head())

   CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          619    France  Female  ...          1               1        101348.88
1          608     Spain  Female  ...          0               1        112542.58
2          502    France  Female  ...          1               0        113931.57
3          699    France  Female  ...          0               0         93826.63
4          850     Spain  Female  ...          1               1         79084.10

[5 rows x 10 columns]


In [4]:
from sklearn.preprocessing import LabelEncoder
inputs['Geography_n']=LabelEncoder().fit_transform(inputs['Geography'])
inputs['Gender_n']=LabelEncoder().fit_transform(inputs['Gender'])
inputs_n=inputs.drop(['Geography','Gender'],axis='columns')
print(inputs_n.head())

   CreditScore  Age  Tenure  ...  EstimatedSalary  Geography_n  Gender_n
0          619   42       2  ...        101348.88            0         0
1          608   41       1  ...        112542.58            2         0
2          502   42       8  ...        113931.57            0         0
3          699   39       1  ...         93826.63            0         0
4          850   43       2  ...         79084.10            2         0

[5 rows x 10 columns]


In [5]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [6]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
import time
classifier=xgboost.XGBClassifier()

random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
start_time = time.time()
random_search.fit(inputs_n,target)
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
--- 22.68133306503296 seconds ---


In [7]:
print(random_search.best_estimator_)

XGBClassifier(colsample_bytree=0.7, gamma=0.3, learning_rate=0.05, max_depth=5,
              min_child_weight=7)


In [8]:
print(random_search.best_params_)

{'min_child_weight': 7, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.7}


In [9]:
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=7, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [10]:
from sklearn.model_selection import cross_val_score
start_time = time.time()
score=cross_val_score(classifier,inputs_n,target,cv=10)
print(score)
print("--- %s seconds ---" % (time.time() - start_time))

[0.874 0.863 0.873 0.865 0.864 0.853 0.859 0.875 0.858 0.856]
--- 6.978029251098633 seconds ---


In [11]:
print(score.mean())

0.8640000000000001


In [13]:
from sklearn.metrics import accuracy_score
predictions=classifier.predict(inputs_n)
accuracy = accuracy_score(target, predictions)

In [12]:
from sklearn.metrics import accuracy_score
classifier=xgboost.XGBClassifier()
model=classifier.fit(inputs_n,target)
predictions=model.predict(inputs_n)
accuracy = accuracy_score(target, predictions)
print(accuracy)

0.8687
